#

                                              BAHIR DAR UNIVERSITY
                                         BAHIR DAR INSTITUTE OF TECHNOLOGY
                                 FACULTY OF COMPUTING DEPARTMENT OF COMPUTER SCIENCE
                            PART OF SPEECH TAGGING IN AMHARIC LANGUAGE BY USING REAR MODELS
                                                    OF
                                        MACHINE LEARNING PROJECT

                                                             Submitted to Abdulkerim M. (Ph.D)
                                                               Done by Abebe Desie (BDU1402211)

                                                                          Bahirdar, Ethiopia
                                                                                 June 2022



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
#print('Tensorflow version: ', tf.__version__)
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
import re

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/2700.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

#df = pd.concat([df_train, df_test , df_dev])

In [ ]:
df

,Text,Label
0,ቬሮኒካ መንገድ ምግብ ሲሸታት የሸተታት ትገባና ቓጥ ይቅርታ ሪሃና ነብሰጡ...,Non-racist
1,ቪቫ ከናንተ በወገን ደም መሸቀጥ ስላል ፈለገ,Non-racist
2,ፖሊስ በጀዋር መሃመድ መኖሪያ ህጋዊ ፈቃድ የሌለው ታጣፊ ታንክ በኪስ መያ...,Non-racist
3,ፖለትካ በሃይማኖት ውሰጥ ማንን እንመን አንደ ክላሽ ጠበጃ መንጫ ትላላቹ ...,Country
4,ፌሚኒዝም ከወንዶች አንፃር መቀንቀን የለበትም የእናቱ ሚስቱ እህቱና ልጁ ...,Non-racist
...,...,...
2731,ሃይ የማይገናኝ አታውራ ታድያ ሰርቶ እንጅ የተከፈለው ሃይማኖቱን ሽጠው ከ...,Non-racist
2732,ሃዋርያት በሰሯት በዘር በቋንቋ በማትከፋፈል በአንድት ሃይማኖት ኦርቶዶክስ...,Non-racist
2733,ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች አበሻ ሴት...,Country
2734,ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች,Country


In [ ]:
import pandas as pd
import re

def expand_amharic_abbreviations(text):
    abbreviation_mapping = {
        'ሀ/ማርያም': 'ሀይለ ማርያም',
        'ሃ/ማርያም': 'ሃይለ ማርያም',
        'ሃ/ማርያምም': 'ሃይለ ማርያም',
        'ህ/ሰብ': 'ህብረተሰብ',
        'ህ/ቁ': 'ህዝባር ቁጥር',
        'ህ/ተሰብ': 'ህብረተሰብ',
        'ህብረት/ኢዴህሕ': 'የኢትዮጵያ ዲሞክራሳዊ ሃይሎች ህብረት',
        'ለ/አ/አ/ድ': 'ለአፍሪካ አንድነት ድርጅት',
        'ለሜ/ጄኔራል': 'ለሜጀር ጄኔራል',
        'ለም/ቤቱ': 'ለምክር ቤቱ',
        'ለም/ቤት': 'ለምክር ቤት',
        'ለምስ/መካ': 'ለምስራቅና መካከለኛ',
        'ለብ/ጄ/ል': 'ለብርጋዴል ጄኔራል',
        'ለተ/መ/ድ': 'ለተባበሩት መንግስታት ድርጅት',
        'ለአ/አ': 'ለአዲስ አበባ',
        'ለአ/አበባ': 'ለአዲስ አበባ',
        'ለአቶ/ወ/ሮ/ወ/ት': 'ለአቶ/ወይዘሮ/ወይዘሪት',
        'ለኢ/ን/ምክር': 'ለኢትዮጵያ ንግድ ምክር',
        'ለኢ/ኦ/ተ/ቤተ': 'ለኢትዮጵያ ኦርቶዶክስ ተዋህዶ ቤተ',
        'ለወ/ሮ': 'ለወይዘሮ',
        'ለወ/ት': 'ለወይዘሪት',
        'ለዶ/ር': 'ለዶክርተር',
        'ዓ.ም': 'አመተ ምህረት',

    }

    for abbreviation, expansion in abbreviation_mapping.items():
        text = text.replace(abbreviation, expansion)

    return text

# Assuming you have a CSV file with a column named 'Amharic_Text'
input_csv_path = '/content/drive/MyDrive/CSV-Data.csv'
output_csv_path = 'expanded_dataset.csv'

# Load the CSV dataset
df= pd.read_csv('/content/drive/MyDrive/2700.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

# Apply abbreviation expansion to the 'Amharic_Text' column
df['Expanded_Amharic_Text'] = df['Text'].apply(expand_amharic_abbreviations)

# Save the expanded dataset to a new CSV file
# Save the normalized dataset to a new CSV file
df.to_csv(output_csv_path, index=False)

print(f"abbreviations dataset saved to {output_csv_path}")


abbreviations dataset saved to expanded_dataset.csv


In [ ]:
class AmharicAbbreviationExpansion:
  def AbbreviationExpansion(train_normalize):
    amh_abbreviation_word ={'ሀ/ማርያም':'ሀይለ ማርያም','ሃ/ማርያም':'ሃይለ ማርያም','ሃ/ማርያምም':'ሃይለ ማርያም','ህ/ሰብ':'ህብረተሰብ','ህ/ቁ':'ህዝባር ቁጥር','ህ/ተሰብ':'ህብረተሰብ','ህብረት/ኢዴህሕ':'የኢትዮጵያ ዲሞክራሳዊ ሃይሎች ህብረት','ለ/አ/አ/ድ':'ለአፍሪካ አንድነት ድርጅት','ለሜ/ጄኔራል':'ለሜጀር ጄኔራል','ለም/ቤቱ':'ለምክር ቤቱ','ለም/ቤት':'ለምክር ቤት','ለምስ/መካ':'ለምስራቅና መካከለኛ','ለብ/ጄ/ል':'ለብርጋዴል ጄኔራል','ለተ/መ/ድ':'ለተባበሩት መንግስታት ድርጅት','ለአ/አ':'ለአዲስ አበባ','ለአ/አበባ':'ለአዲስ አበባ','ለአቶ/ወ/ሮ/ወ/ት':'ለአቶ/ወይዘሮ/ወይዘሪት','ለኢ/ን/ምክር':'ለኢትዮጵያ ንግድ ምክር','ለኢ/ኦ/ተ/ቤተ':'ለኢትዮጵያ ኦርቶዶክስ ተዋህዶ ቤተ','ለወ/ሮ':'ለወይዘሮ','ለወ/ት':'ለወይዘሪት','ለዶ/ር':'ለዶክርተር','ለጠ/ሚ':'ለጠቅላይ ሚኒስቴር','ለጠ/ሚ/መለስ':'ለጠቅላይ ሚኒስቴር መለስ','ለጠ/ሚ/ሩ':'ለጠቅላይ ሚኒስቴሩ','ለጠ/ሚ/ሩና':'ለጠቅላይ ሚኒስትሩና','ለጠ/ሚ/ር':'ለጠቅላይ ሚኒስቴር','ለጠ/ሚኒስትር':'ለጠቅላይ ሚኒስትር','ለጠ/ሚኒስትርነት':'ለጠቅላይ ሚኒስትርነት','ለጠ/ፍ/ቤት':'ለጠቅላይ ፍርድ ቤት','ለፌ/ም/ቤት':'ለፌደራል ምክር ቤት','ለፍ/ቤቱ':'ለፍርድ ቤቱ','ለፍ/ቤት':'ለፍርድ ቤት','ለፕ/ት':'ለፕሬዚዳንት','ሊ/መንበር':'ሊቀ መንበር','ሊ/መንበርና':'ሊቀ መንበርና','ሊማትድ/ካናዳ':'ሊሚትድ','ሌ/ኮ':'ሌተናል ኮሎኔል','ሌ/ኮ/ል':'ሌተናል ኮሎኔል','ሌ/ኮል':'ሌተናል ኮሎኔል','ሌ/ኮሎኔል':'ሌተናል ኮሎኔል','ሌ/ኰ/ል':'ሌተናል ኮሎኔል','ሌ/ኰ/በየነ':'ሌተናል ኮሎኔል በየነ','ሌ/ጄ':'ሌተናል ጄነራል','ሌ/ጄ/ል':'ሌተናል ጄነራል','ሌ/ጄ/ተስፋዬ':'ሌተናል ጄነራል ተስፋዬ','ሌ/ጄኔራል':'ሌተናል ጄኔራል','ሕ/ሥ/ሥ/ቁ':'ሕግ ሥነ ሥርዓት ቁጥር','መ/ሚ':'መከላከያ ሚኒስቴር','መ/ሚኒስትር':'መከላከያ ሚኒስትር','መ/ም/ቤት':'መስተዳድር ምክር ቤት','መ/ቤት':'መስሪያ ቤት','መ/ቤቶቹ':'መስሪያ ቤቶቹ','መ/ቤቶች':'መስሪያ ቤቶች','መ/አ':'መቶ አለቃ','መ/አ/ግርማ':'መቶ አለቃ ግርማ','መ/አለቃ':'መቶ አለቃ','መርዕድጦቢያጋዜጣና/መጽ':'ጋዜጣና መጽሔት','ሚ/ሚ/ር':'ሚኒስቴር ሚኒስትር','ሚ/ም/ቤት':'ሚኒስቴር ምክር ቤት','ሚ/ሩ':'ሚኒስትሩ','ሚ/ር':'ሚኒስቴር','ማ/ም':'ማህበር ምክር','ማ/ቡ':'ማህበሩ','ማ/ቤት':'ማረፊያ ቤት','ማ/ኮ':'ማዕከላዊ ኮሚቴ','ማ/ኮ/አባልና':'ማዕከላዊ ኮሚቴ አባልና','ማ/ኮሚቴው':'ማዕከላዊ ኮሚቴው','ኢ/ነ/ጋ/ማ/ማህበር':'የኢትዮጵያ ነፃ ጋዜጠኞች ማህበር','ኢነጋማ':'የኢትዮጵያ ነፃ ጋዜጠኞች ማህበር','ሜ/ር':'ሜጀር ጄኔራል','ሜ/ሻለቃ':'ሜጀር ጄነራል ሻለቃ','ሜ/ጄ':'ሜጀር ጄኔራል','ሜ/ጄ/ል':'ሜጀር ጄኔራል','ሜ/ጄ/ልነት':'ሜጀር ጄኔራልነት','ሜ/ጄ/ጻድቃን':'ሜጀር ጄኔራል ጻድቃን','ም/ሊ/መ':'ምክትል ሊቀ መንበር','ም/ሊ/መንበሩ':'ምክትል ሊቀ መንበሩ','ም/ሊ/መንበር':'ምክትል ሊቀ መንበር','ም/ሊቀ':'ምክትል ሊቀ','ም/ሊቀመንበር':'ምክትል ሊቀመንበር','ም/ሊቀመንበርና':'ምክትል ሊቀመንበርና','ም/መ/አ':'ምክትል መቶ አለቃ','ም/መ/አለቃ':'ምክትል መቶ አለቃ','ም/መ/አለቃና':'ምክትል መቶ አለቃና','ም/መቶ/አለቃ':'ምክትል መቶ አለቃ','ም/ሚ/ር':'ምክትል ሚኒስቴር','ም/ሚኒስትር':'ምክትል ሚኒስትር','ም/ሥ/አስኪያጅ':'ምክትል ሥራ አስኪያጅ','ም/ሥ/አስኪያጅና':'ምክትል ሥራ አስኪያጅና','ም/ሳጅን':'ምክትል ሳጅን','ም/ቤቱ':'ምክር ቤቱ','ም/ቤቱን':'ምክር ቤቱን','ም/ቤታችን':'ምክር ቤታችን','ም/ቤት':'ምክር ቤት','ም/ቤትም':'ምክር ቤትም','ም/ቤትና':'ምክር ቤትና','ም/ቤትን':'ምክር ቤትን','ም/ቤቶች':'ምክር ቤቶች','ም/ተ/ሊቀመንበር':'ምክትል ተጠባባቂ ሊቀመንበር','ም/አስተዳዳሪዎችን':'ምክትል አስተዳዳሪዎችን','ም/አዘጋጁ':'ምክትል አዘጋጁ','ም/አዛዥ':'ምክትል አዛዥ','ም/አፈጉባኤ':'ምክትል አፈጉባኤ','ም/ኮሚሽነር':'ምክትል ኮሚሽነር','ም/ዋ/ፀሐፊ':'ምክትል ዋና ፀሐፊ','ም/ዋና':'ምክትል ዋና','ም/ዋና/ፀሐፊ':'ምክትል ዋና ፀሐፊ','ም/ዋን':'ምክትል ዋና','ም/ው/ጉ/ሚ':'ምክትል ውጭ ጉዳይ ሚኒስትር','ም/ው/ጉዳይ':'ምክትል ውጭ ጉዳይ','ም/የውጪ/ጉ/ሚ/ር':'ምክትል የውጪ ጉዳይ ሚኒስቴር','ም/የፍትህ':'ምክትል የፍትህ','ም/ጠ/ሚ':'ምክትል ጠቅላይ ሚኒስትር','ም/ጠ/ሚ/ሩ':'ምክትል ጠቅላይ ሚኒስትሩ','ም/ጠ/ሚ/ር':'ምክትል ጠቅላይ ሚኒስትር','ም/ጠ/ሚ/ርነት':'ምክትል ጠቅላይ ሚኒስትርነት','ም/ጠ/ሚ/ርና':'ምክትል ጠቅላይ ሚኒስትርና','ም/ጠ/ሚርና':'ምክትል ጠቅላይ ሚኒስትርና','ም/ጠ/ሚኒስትሩ':'ምክትል ጠቅላይ ሚኒስትሩ','ም/ጠ/ሚኒስትር':'ምክትል ጠቅላይ ሚኒስትር','ም/ጠ/ሚኒስትርነት':'ምክትል ጠቅላይ ሚኒስትርነት','ም/ጠ/ሚኒስትርና':'ምክትል ጠቅላይ ሚኒስትርና','ም/ጠ/ሚንስትር':'ምክትል ጠቅላይ ሚንስትር','ም/ጠቅላይ':'ምክትል ጠቅላይ','ም/ጸሐፊ':'ምክትል ጸሐፊ','ም/ፀሐፊው':'ምክትል ፀሐፊው','ም/ፕ/ት':'ምክትል ፕሬዚዳንት','ም/ፕ/ዋና':'ምክትል ፕሬዚዳንት ዋና','ም/ፕሬዘዳንት':'ምክትል ፕሬዘዳንት','ም/ፕሬዚዳንት':'ምክትል ፕሬዚዳንት','ም/ፕሬዝዳንት':'ምክትል ፕሬዝዳንት','ም/ፕሬዝዳንትነት':'ምክትል ፕሬዝዳንትነት','ምጠ/ሚ/ር':'ምክትል ጠቅላይ ሚኒስትር','ሥ/ሥ/ቁ':'ሥነ ሥርዓት ቁጥር','ሥ/አ/ኮ':'ሥራ አስፈፃሚ ኮሚቴ','ሥ/አስኪያጅ':'ሥራ አስኪያጅ','ሥ/አስኪያጆችን':'ሥራ አስኪያጆችን','ሥ/አስፈፃሚ':'ሥራ አስፈፃሚ','ረ/ሳጅን':'ረዳት ሳጅን','ረ/ፕ/ር':'ረዳት ፕሮፌሰር','ረ/ፕሮፌሰር':'ረዳት ፕሮፌሰር','ሰዓትም/ሊ/መንበር':'ሰዓትም ሊቀ መንበር','ሲከሽፍ/ሰራ':'ሲከሽፍ/ሰራ','ስ/አ/ኮ':'ስራ አስፈጻሚ ኮሚቴ','ሻ/ል':'ሻምበል','ቀ/ች':'ቀኝአዝማች','ቀ/ነቀአ':'ቀኝአዝማች ነቅአ','ቀ/ገ/ማ':'ቀበሌ ገበሬ ማህበር','ቀ/ገ/ማኅበራት':'ቀበሌ ገበሬ ማኅበራት','ቀ/ገ/ማኅበር':'ቀበሌ ገበሬ ማኅበር','ቅ/ሚካኤል':'ቅዱስ ሚካኤል','ቅ/ድንግል':'ቅድስት ድንግል','ቅ/ጳውሎስ':'ቅዱስ ፓውሎስ','ቅ/ጽ':'ቅርንጫር ጽህፈት ቤቶች','ቅ/ጽ/ቤቱን':'ቅርንጫር ጽህፈት ቤቱን','ቅ/ጽ/ቤት':'ቅርንጫር ጽህፈት ቤት','ቅ/ጽ/ቤቶች':'ቅርንጫር ጽህፈት ቤቶች','ቅ/ጽ/ቤቶችና':'ቅርንጫር ጽህፈት ቤቶችና','ቅ/አ':'ቅሬታ አስተያየት','በ/መ/ሕ/ሥ/ሥ':'በመተዳደሪያ ሕግ ሥነ ሥርዓት','በ/መሕ/ሥ/ሥ/ቁ':'በመተዳደሪያ ሕግ ሥነ ሥርዓት ቁጥር','በ/ተ/መ/ድ':'በተባበሩት መንግስታት ድርጅት','በመ/ቤቴ':'በመስሪያ ቤቴ','በማ/ኮሚቴ':'በማዕከላዊ ኮሚቴ','በም/ሊቀመንበርነት':'በምክትል ሊቀመንበርነት','በም/ቤቱ':'በምክር ቤቱ','በም/ቤት':'በምክር ቤት','በም/ክ':'በምክትል','በም/ጠ/ሚኒስትሩ':'በምክትል ጠቅላይ ሚኒስትሩ','በም/ጠ/ሚኒስትር':'በምክትል ጠቅላይ ሚኒስትር','በም/ጠ/ሚኒስትርነታቸው':'በምክትል ጠቅላይ ሚኒስትርነታቸው','በሥ/አስኪያጅነት':'በሥራ አስኪያጅነት','በረ/ፕ':'በረዳት ፕሮፌሰር','በቀ/ኃ/ሥላሴ':'በቀዳማዊ ኃይለ ሥላሴ','በቤ/ክ/ልጆች':'በቤተክርስቲያን ልጆች','በተ/መ/ድ':'በተባበሩት መንግስታት ድርጅት','በት/ቤቶች':'በትምህርት ቤቶች','ፕ/ር':'ፕሮፌሰር','በነፕ/ር':'በነፕሮፌሰር','በአ/አ':'በአዲስ አበባ','በአ/አ/ዩኒቨርስቲ':'በአዲስ አበባ ዩኒቨርስቲ','በአ/አ/ድ':'በአፍሪካ አንድነት ድርጅት','በአ/አ/ድርጅት':'በአፍሪካ አንድነት ድርጅት','በአ/አ/ድና':'በአፍሪካ አንድነት ድርጅትና','በአ/አበባ':'በአዲስ አበባ','በአ/ኦ/ተ':'በኢትዮጵያ ኦርቶዶክስ ተዋህዶ','በኢ/ት/ኦ/ተ/ቤ/ክ/ሥርዓተ':'በኢትዮጵያ ኦርቶዶክስ ተዋህዶ ቤተክርስቲያን ሥርዓተ','ዶ/ር':'ዶክተር','በእነዶ/ር':'በእነዶክተር','በወ/ሕ/ቁ':'በወንጀለኛ መቅጫ ሕግ ቁጥር','በወ/መ/ህ':'በወንጀለኛ መቅጫ ሕግ','በወ/መ/ህ/ስ/ስ':'በወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት','በወ/መ/ሕ/ሥ/ሥ/ቁ':'በወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት ቁጥር','በወ/መ/ሕ/ሥ/ቁ':'በወንጀለኛ መቅጫ ሕግ ሥርዓት ቁጥር','በወ/መ/ሕ/ቁ':'በወንጀለኛ መቅጫ ሕግ ቁጥር','በወ/መ/ሕ/ቁጥር':'በወንጀለኛ መቅጫ ሕግ ቁጥር','በወ/መ/ሕግ':'በወንጀለኛ መቅጫ ሕግ','በወ/መ/ሕግ/ሥ/ሥ':'በወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት','በወ/መ/ሕግ/ሥ/ሥ/ቁጥር':'በወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት ቁጥር','በወ/መ/ሥ/ህ':'በወንጀለኛ መቅጫ ሥርዓት ህግ','በወ/መ/ሥ/ሥ':'በወንጀለኛ መቅጫ ሥነ ሥርዓት','በወ/መ/ሥ/ሥ/ሕ/ቁ':'በወንጀለኛ መቅጫ ሥነ ሥርዓት ሕግ ቁጥር','በወ/መ/ሥ/ሥ/ቁ':'በወንጀለኛ መቅጫ ሥነ ሥርዓት ቁጥር','በወ/መ/ሥ/ቁ':'በወንጀለኛ መቅጫ ሥርዓት ቁጥር','በወ/መ/ሰ/ቁጥር':'በወንጀለኛ መቅጫ ሥርዓት ቁጥር','በወ/መ/ቁ':'በወንጀለኛ መቅጫ ቁጥር','በወ/መህ/ቁ':'በወንጀለኛ መቅጫ ህግ ቁጥር','በወ/መሕ/ሥ/ሥ/ቁ':'በወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት ቁጥር','በወ/መሕ/ቁጥር':'በወንጀለኛ መቅጫ ሕግ ቁጥር','በወ/መዝ/ቁ':'በወንጀል መዝገብ ቁጥር','በወ/ሪት':'በወይዘሪት','በወ/ሮ':'በወይዘሮ','በወ/ት':'በወይዘሪት','በወ/ይግ/መዝ/ቁ':'በወንጀል ይግባኝ መዝገብ ቁጥር','በወ/መዝገብ':'በወንጀል መዝገብ','በው/ጉ/ሚር':'በውጪ ጉዳይ ሚኒስቴር','በደ/አፍሪካ':'በደቡብ አፍሪካ','በዶ/ሩ':'በዶክተሩ','በዶ/ር':'በዶክተር','በጠ/ሚ/ሩ':'በጠቅላይ ሚኒስትሩ','በጠ/ሚ/ሩና':'በጠቅላይ ሚኒስትሩና','በጠ/ሚ/ር':'በጠቅላይ ሚኒስትር','በጠ/ሚ/ር/ነት':'በጠቅላይ ሚኒስትርነት','በጠ/ሚ/ሮ':'በጠቅላይ ሚኒስትር','በጠ/ሚ/ቢሮ':'በጠቅላይ ሚኒስትር ቢሮ','በጠ/ሚ/ጽ/ቤት':'በጠቅላይ ሚኒስቴር ጽህፈት ቤት','በጠ/ሚኒ/ሩ':'በጠቅላይ ሚኒስትሩ','በጠ/ቤ/ክህነት':'በጠቅላይ ቤተ ክህነት','በጠ/ፍ/ቤት':'በጠቅላይ ፍርድ ቤት','በጽ/ቤቱ':'በጽህፈት ቤቱ','በጽ/ቤታቸው':'በጽህፈት ቤታቸው','በፅ/ቤታቸው':'በፅህፈት ቤታቸው','በፈ/ከ/ፍ/ቤት':'በፌደራል ከፍተኛ ፍርድ ቤት','በፍ/ቤቱ':'በፍርድ ቤቱ','በፍ/ቤቱና':'በፍርድ ቤቱና','በፍ/ቤት':'በፍርድ ቤት','በፍ/ቤቶችና':'በፍርድ ቤቶችና','በፍ/ብ/ሥ/ሥ/ህ/ቁ':'በፍህታ ብሄር ሥነ ሥርዓት ህግ ቁጥር','በፍ/ብ/ሥ/ሥ/ሕ/ቁ':'በፍህታ ብሄር ሥነ ሥርዓት ሕግ ቁጥር','በፍ/ብ/ሥ/ሥ/ሕ/ቁጥር':'በፍህታ ብሄር ሥነ ሥርዓት ሕግ ቁጥር','በፍ/ብ/ስ/ሥ':'በፍህታ ብሄር ስነ ሥርዓት','በፍብ/ሥ/ሥ/ሕ/ቁ':'በፍህታ ብሄር ሥነ ሥርዓት ሕግ ቁጥር','ቤ/ክ':'ቤተክርስቲያን','ቤ/ክህነት':'ቤተክህነት','ብ/ል/ጄ':'ብርጋዴል ጄነራል','ብ/ብ/ሕ':'ብሄር ብሄረሰቦችና ሕዝቦች','ብ/ብ/ሕ/ክ':'ብሄር ብሄረሰቦችና ሕዝቦች ክልል','ብ/ብ/ክልል':'ብሄር ብሄረሰቦች ክልል','ብ/ጀኔራል':'ብርጋዴል ጄነራል','ብ/ጄ':'ብርጋዴል ጄኔራል','ብ/ጄ/ል':'ብርጋዴል ጄኔራል','ብ/ጄ/ሳሞራ':'ብርጋዴል ጄኔራል ሳሞራ','ብ/ጄ/ተፈራ':'ብርጋዴል ጄኔራል ተፈራ','ብ/ጄል':'ብርጋዴል ጄኔራል','ብ/ጄኔራል':'ብርጋዴል ጄኔራል','ብ/ጄኔራሎች':'ብርጋዴል ጄኔራሎች','ተ/ሃይማኖት':'ተክለ ሃይማኖት','ተ/ም':'ተጠባባቂ ምክትል','ተ/ም/ሊቀ':'ተጠባባቂ ምክትል ሊቀ','ተ/ም/ሊቀመንበር':'ተጠባባቂ ምክትል ሊቀመንበር','ተ/ም/ኘሬዚዱንቱ':'ተጠባባቂ ምክትል ፕሬዚዳንቱ','ተ/ም/ኘሬዚዱንት':'ተጠባባቂ ምክትል ፕሬዚዳንቱ','ተ/ም/ኮሚሽነር':'ተጠባባቂ ምክትል ኮሚሽነር','ተ/ም/ፕሬዚዱንቱ':'ተጠባባቂ ምክትል ፕሬዚዳንቱ','ተ/ም/ፕሬዚዳንቱ':'ተጠባባቂ ምክትል ፕሬዚዳንቱ','ተ/ም/ፕሬዝዳንቱ':'ተጠባባቂ ምክትል ፕሬዚዳንቱ','ተ/ም/ፕሬዝዳንት':'ተጠባባቂ ምክትል ፕሬዚዳንት','ተ/እግዚ':'ተክለእግዚ','ተ/ዋና':'ተጠባባቂ ዋና','ተሾሙ/ወረዱ':'ተሾሙ ወረዱ','ተቀ/ም/ፕሬዚዳ':'ተቀዳሚ ምክትል ፕሬዚዳንት','ት/ቤቱን':'ትምህርት ቤቱን','ት/ቤታቸው':'ትምህርት ቤታቸው','ት/ቤት':'ትምህርት ቤት','ት/ቤቶች':'ትምህርት ቤቶች','ት/ክፍል':'ትምህርት ክፍል','ኃ/መስቀል':'ኃይለ መስቀል','ኃ/ሚካኤል':'ኃይለ ሚካኤል','ኃ/ማሪያም':'ኃይለ ማሪያም','ኃ/ማርያም':'ኃይለ ማርያም','ኃ/ማርያምን':'ኃይለ ማርያምን','ኃ/ሥላሴ':'ኃይለ ሥላሴ','ኃ/ስላሴ':'ኃይለ ስላሴ','ኃ/የ/የግል':'ኃላፊነቱ የተወሰነ የግል','ኃ/የተ/የግ/ማ':'ኃላፊነቱ የተወሰነ የግል ማህበር','ኃ/የተ/የግ/ማህበር':'ኃላፊነቱ የተወሰነ የግል ማህበር','ኃ/የተ/የግል':'ኃላፊነቱ የተወሰነ የግል','ኃ/የተ/የግል/ማኀብር':'ኃላፊነቱ የተወሰነ የግል ማህብር','ኃ/የተ/ግ/ማ':'ኃላፊነቱ የተወሰነ የግል ማህበር','ኃ/ገብርኤል':'ኃይለ ገብርኤል','ኃ/ጊዎርገስ':'ኃይለ ጊዎርገስ','ኃ/ጊዮርጊስ':'ኃይለ ጊዮርጊስ','ኃላ/የተ/የግ/ማኅበር':'ኃላፊነቱ የተወሰነ የግል ማህበር','ነው/ናት':'ነው ናት','ን/ም/ቤት':'ንግድ ምክር ቤት','ን/ም/ቤቶች':'ንግድ ምክር ቤቶች','አ/ማ':'አገልግሎት ማህበር','አ/አ':'አዲስ አበባ','አ/አ/ድም':'አፍሪካ አንድነት ድርጅትም','አ/አበባ':'አዲስ አበባ','አ/ኢ':'አዲስ አበባ','ኢ/ሠ/ማ':'የኢትዮጵያ ሰራተኛ ማህበር','ኢ/ር':'ኢንጂነር','ኢር/ግዛቸው':'ኢንጂነር ግዛቸው','ኢተፖድኅ/አብኮ':'የኢትዮጵያ ተቃዋሚ ፖለቲካ ድርጅቶች ኅብረት','ኢትዮጵያዊ/ኢትዮጵያዊት':'ኢትዮጵያዊ/ኢትዮጵያዊት', 'ኢዲዩ/ኢዴኅፓ':'የኢትዮጵያ ዲሞክራሲያዊ ዩኒቲ/የኢትዮጵያ ዴሞክራሲያዊ ኅብረት ፓርቲ','ኢዴኅፓ/ኢድዩ':'የኢትዮጵያ ዴሞክራሲያዊ ኅብረት ፓርቲ/የኢትዮጵያ ዲሞክራሲያዊ ዩኒቲ','ኤ/ማ/ሹም':'ኤታማዡር ሹም','እ/ኤ/አ':'እንደ ኤሮፓውያን አቆጣጠር','እ/ጉ/ም/ቤት':'እስልምና ጉዳይ ምክር ቤት','እነጄ/ል':'እነጄነራል','እፅ/ሄሮይን':'እፅ/ሄሮይን','ኦ/ቤተ':'ኦርቶዶክስ ቤተክርስቲያን','ኦ/ተ/ቤ/ክርስቲያን':'ኦርቶዶክስ ተዋህዶ ቤተክርስቲያን','ኦ/ተ/ቤተ':'ኦርቶዶክስ ተዋህዶ ቤተክርስቲያን','ኦ/ተ/እምነትና':'ኦርቶዶክስ ተዋህዶ እምነትና','ከ/ፍ/ቤት':'ከፍተኛ ፍርድ ቤት','ከመ/አለቃ':'ከመቶ አለቃ','ከማ/ኮ':'ከማዕከላዊ ኮሚቴ','ከማ/ኮ/ው':'ከማዕከላዊ ኮሚቴው','ከም/መ/አለቃ':'ከምክትል መቶ አለቃ','ከም/ቤቱ':'ከምክር ቤቱ','ከም/ቤት':'ከምክር ቤት','ከም/ጠ/ሚኒስትር':'ከምክትል ጠቅላይ ሚኒስቴር','ከቀኛ/ዝማች':'ከቀኝአዝማች','ከቅ/ማርያም':'ከቅድስት ማርያም','ከነዶ/ር':'ከነዶክተር','ከአ/አበባ':'ከአዲስ አበባ','ከኮ/ል':'ከኮሎኔል','ከወ/ሮ':'ከወይዘሮ','ከው/ጉ/ሚ':'ከውጭ ጉዳይ ሚኒስትር','ከው/ጉ/ሚ/ር':'ከውጭ ጉዳይ ሚኒስትር','ከዶ/ር':'ከዶክተር','ከጠ/ሚ':'ከጠቅላይ ሚኒስትር','ከጠ/ሚ/ልዩ':'ከጠቅላይ ሚኒስትር ልዩ','ከጠ/ሚ/መለስ':'ከጠቅላይ ሚኒስትር መለስ','ከጠ/ሚ/ሩ':'ከጠቅላይ ሚኒስትሩ','ከጠ/ሚ/ር':'ከጠቅላይ ሚኒስትር','ከጠ/ሚ/ርነት':'ከጠቅላይ ሚኒስትርነት','ከጠ/ሚኒስትር':'ከጠቅላይ ሚኒስትር','ከጠ/ሚንስትሩ':'ከጠቅላይ ሚኒስትሩ','ከጠ/ም/ቤት':'ከጠቅላይ ምክር ቤት','ከፌ/ፖ/ወንጀል':'ከፌደራል ፖሊስ ወንጀል','ከፍ/ቤቱ':'ከፍርድ ቤቱ','ከፍ/ቤት':'ከፍርድ ቤት','ከፍ/ፍ/ቤት':'ከፍተኛ ፍርድ ቤት','ከፕ/ት':'ከፕሬዚዳንት','ኪ/ማርያም':'ኪዳነ ማርያም','ክ/ሀገር':'ክፍለ ሀገር','ክ/ከ':'ክፍለ ከተማ','ክ/ከተማ':'ክፍለ ከተማ','ክልል/ዞን':'ክልል ዞን','ኮ/ሉ':'ኮሎኔሉ','ኮ/ሌ':'ኮለኔል','ኮ/ል':'ኮሎኔል','ኮ/ሎ':'ኮሎኔል','ኰ/ሌ':'ኰሎኔል','ወ/ሐና':'ወልደ ሐና','ወ/መ/ህግና':'ወንጀለኛ መቅጫ ህግና','ወ/መ/ሕ/ሥ/ቁጥር':'ወንጀለኛ መቅጫ ሕግ ሥነሥርዓት ቁጥር','ወ/መ/ሕ/ቁ':'ወንጀለኛ መቅጫ ሕግ ቁጥር','ወ/መ/ሕግ':'ወንጀለኛ መቅጫ ሕግ','ወ/ሚካኤል':'ወልደ ሚካኤል','ወ/ማሪያም':'ወልደ ማሪያም','ወ/ማርያም':'ወልደ ማርያም','ወ/ሥላሴ':'ወልደ ሥላሴ','ወ/ሪት':'ወይዘሪት','ወ/ር':'ወታደር','ወ/ሮ':'ወይዘሮ','ወ/ሮ/ት':'ወይዘሮ/ወይዘሪት','ወ/ሮዋም':'ወይዘሮዋም','ወ/ሰማያት':'ወልደ ሰማያት','ወ/ሰማያትን':'ወልደ ሰማያትን','ወ/ስላሴ':'ወልደ ስላሴ','ወ/ቂርቆስ':'ወልደ ቂርቆስ','ወ/ቂርቆስጦቢያጋዜጣና/መጽ':'ወልደ ቂርቆስ ጦቢያ ጋዜጣና መጽሔት','ወ/ተንሣይ':'ወልደ ተንሣይ','ወ/ተንሳይ':'ወልደ ተንሳይ','ወ/ት':'ወይዘሪት','ወ/ኪሮስ':'ወልደ ኪሮስ','ወ/የስ':'ወልደ የስ','ወ/የስን':'ወልደ የስን','ወ/ዮሐንስ':'ወልደ ዮሐንስ','ወ/ገሪማ':'ወልደ ገሪማ','ወ/ገብርኤል':'ወልደ ገብርኤል','ወ/ጊዮርጊስ':'ወልደ ጊዮርጊስ','ወ/ፍ/ቤት':'ወረዳ ፍርድ ቤት','ዋ/ሳጅን':'ዋና ሳጅን','ው/ጉ/ሚ':'ውጭ ጉዳይ ሚኒስቴር','ው/ጉ/ሚ/ር':'ውጭ ጉዳይ ሚኒስቴር','ው/ጉ/ሚኒስቴር':'ውጭ ጉዳይ ሚኒስቴር','ው/ጉ/ሚኒስትሩ':'ውጭ ጉዳይ ሚኒስትሩ','ው/ጉ/ሚኒስትር':'ውጭ ጉዳይ ሚኒስትር','ዐ/ሕግ':'ዐቃቤ ሕግ','ዓ/ም':'ዓመተ ምህረት','ዕ/ቤቶች':'ፅህፈት ቤቶች','ዕ/ቤ':'ፅህፈት ቤት','የ/ማ/ኮሚቴ':'የማዕከላዊ ኮሚቴ','የ/ጠ/ም/ቤቱ':'የጠቅላይ ምክር ቤቱ','የል/ጀ/ል':'የሌተናል ጀነራል','የመ/አለቃ':'የመቶ አለቃ','የሚ/መ/ቤት':'የሚኒስቴር መስሪያ ቤት','የሚ/ር':'የሚኒስቴር','የማ/ቤቱ':'የማረሚያ ቤቱ','የማ/ኮሚቴ':'የማዕከላዊ ኮሚቴ','የሜ/ጄ':'የሜጄር ጄኔራል','የሜ/ጄ/ል':'የሜጄር ጄኔራል','የሜ/ጄ/ልነት':'የሜጄር ጄኔራልነት','የሜ/ጄኔራልነት':'የሜጄር ጄነራልነት','የም/መ':'የምክትል መቶ','የም/መ/አለቃ':'የምክትል መቶ አለቃ','የም/ቤቱ':'የምክር ቤቱ','የም/ቤቱን':'የምክር ቤቱን','የም/ቤት':'የምክር ቤት','የም/ጠ/ሚ/ሩን':'የምክትል ጠቅላይ ሚኒስትሩን','የስ/አ/ኮሚቴ':'የስራ አስኪያጅ ኮሚቴ','የሻ/ል':'የሻምበል','የቤ/ክ/ኑ':'የቤተክርስቲያኑ','የብ/ጀ/ተፈራ':'የብርጋዴር ጀነራል ተፈራ','የብ/ጄ/ልነት':'የብርጋዴር ጄኔራልነት','የተ/ም/ሊቀ':'የተጠባባቂ ምክትል ሊቀ','የት/ቤቱ':'የትምህርት ቤቱ','አ/ማ':'አማራ','የአ/አ/ዩ':'የአዲስ አበባ ዩኒቨርስቲ','የአ/አ/ድ':'የአፍሪካ አንድነት ድርጅት','የአ/አ/ድርጅት':'የአፍሪካ አንድነት ድርጅት','የአ/አበባ':'የአዲስ አበባ','የአ/አ':'የአዲስ አበባ','የኢ/ን/ም/ቤት':'የኢትዮጵያ ንግድ ምክር ቤት','የኢ/ኦ/ቤ/ክርስቲያን':'የኢትዮጵያ ኦርቶዶክስ ቤተክርስቲያን','የኢ/ኦ/ተ':'የኢትዮጵያ ኦርቶዶክስ ተዋህዶ','የኢ/ኦ/ተ/ቤ/ክ':'የኢትዮጵያ ኦርቶዶክስ ተዋህዶ ቤተክርስቲያን','የኢ/ኦ/ተ/ቤተክርስቲያን':'የኢትዮጵያ ኦርቶዶክስ ተዋህዶ ቤተክርስቲያን','የኢ/ወ/መ/ህ':'የኢትዮጵያ ወንጀለኛ መቅጫ ህግ','የኢህዴን/ብአዴን':'ኢትዮጵያ ህዝቦች ዴሞክራሲያዊ ንቅናቄ/ብሔረ አማራ ዴሞክራሲያዊ ንቅናቄ','የኢት/አት/ፌዴሬሽን':'የኢትዮጵያ አትሌቲክስ ፌዴሬሽን','የኢት/እ/ኳስ':'የኢትዮጵያ እግር ኳስ','የእነዶ/ር':'የእነዶክተር','የኦ/ተ/ቤ/ክ':'የኦርቶዶክስ ተዋህዶ ቤተክርስቲያን','የከ/ፍ/ቤት':'የከፍተኛ ፍ/ቤት','የኮ/ል':'የኮሎኔል','የወ/ሕ/ቁ':'የወንጀለኛ መቅጫ ሕግ ቁጥር','የወ/መ/ህ':'የወንጀለኛ መቅጫ ህግ','የወ/መ/ህ/ቁ':'የወንጀለኛ መቅጫ ህግ ቁጥር','የወ/መ/ሕ':'የወንጀለኛ መቅጫ ሕግ','የወ/መ/ሕ/ሥ':'የወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት','የወ/መ/ሕ/ሥ/ሥ/ቁ':'የወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት ቁጥር','የወ/መ/ሕ/ቁ':'የወንጀለኛ መቅጫ ሕግ ቁጥር','የወ/መ/ሕ/ቁጥር':'የወንጀለኛ መቅጫ ሕግ ቁጥር','የወ/መ/ሕግ':'የወንጀለኛ መቅጫ ሕግ','የወ/መ/ሕግ/ሥ/ሥ/ቁጥር':'የወንጀለኛ መቅጫ ሕግ ሥነ ሥርዓት ቁጥር','የወ/መ/ሥ/ቁ':'የወንጀለኛ መቅጫ ሥነስርዓት ቁጥር','የወ/መ/ቁጥር':'የወንጀለኛ መቅጫ ቁጥር','የወ/መሕ/ቁ':'የወንጀለኛ መቅጫ ሕግ ቁጥር','የወ/መዝገብ':'የወንጀል መዝገብ','የወ/ሮ':'ወይዘሮ','የወ/ት':'ወይዘሪት','የወያኔ/ኢህአዴግ':'የወያኔ/ኢትዮጵያ ህዝባዊ አንድነት ዴሞክራሲያዊ ግንባር','የው/ጉ/ሚ':'የውጪ ጉዳይ ሚኒስቴር','የው/ጉ/ሚ/ሩ':'የውጪ ጉዳይ ሚኒስቴሩ','የው/ጉ/ሚ/ር':'የውጪ ጉዳይ ሚኒስቴር','የው/ጉ/ሚኒስቴር':'የውጪ ጉዳይ ሚኒስቴር','የው/ጉ/ሚኒስትሮች':'የውጪ ጉዳይ ሚኒስትሮች','የው/ጊ/ሚ/ር':'የውጪ ጉዳይ ሚኒስቴር','የደ/ብ/ብ':'የደቡብ ብሄር ብሄረሰብ','የደ/አፍሪካ':'የደቡብ አፍሪካ','የደ/አፍሪካውያን':'የደቡብ አፍሪካውያን','የዶ/ር':'የዶክተር','የዶ/ት':'የዶክተር','የጅቡቲውየውጪ/ጉ/ሚ/ር':'የጅቡቲው የውጪ ጉዳይ ሚኒስቴር','የጠ/ሚ':'የጠቅላይ ሚኒስትር','የጠ/ሚ/መለስ':'የጠቅላይ ሚኒስትር መለስ','የጠ/ሚ/ሩ':'የጠቅላይ ሚኒስትሩ','የጠ/ሚ/ሩና':'የጠቅላይ ሚኒስትሩና','የጠ/ሚ/ሩን':'የጠቅላይ ሚኒስትሩን','የጠ/ሚ/ር':'የጠቅላይ ሚኒስትር','የጠ/ሚ/ቢሮና':'የጠቅላይ ሚኒስትር ቢሮና','የጠ/ሚንስትሩን':'የጠቅላይ ሚንስትሩን','የጠ/ም/ቤት':'የጠቅላይ ምክር ቤት','የጠ/ቤተ':'የጠቅላይ ቤተ','የጠ/ፍ/ቤት':'የጠቅላይ ፍርድ ቤት','የጽ/ቤት':'ጽህፈት ቤት','የፌ/መ/ደረጃ':'የፌደራል መደበኛ ደረጃ','የፌ/ከ/ፍ/ቤት':'የፌደራል ከፍተኛ ፍርድ ቤት','የፌ/ጠ/ፍ/ቤት':'የፌደራል ጠቅላይ ፍርድ ቤት','የፌ/ፍ/ቤት':'የፌደራል ፍርድ ቤት','የፍ/ሥ/ሥ/ሕ/ቁ':'የፍታብሄር ሥነ ሥርዓት ሕግ ቁጥር','የፍ/ቤቱን':'የፍርድ ቤቱን','የፍ/ቤት':'የፍርድ ቤት','የፍ/ብ/ችሎት':'የፍታብሄር ችሎት','የፕ/ር':'የፕሬዚዳንት','የፕ/ት':'የፕሬዚዳንት','ይችላልው/ጉ/ሚ':'ይችላል ውጭ ጉዳይ ሚኒስቴር','ደ/አፍሪካ':'ደቡብ አፍሪካ','ዶ/ር':'ዶክተር','ጄ/ል':'ጄኔራል','ገ/ሂወትና':'ገብረ ሂወትና','ገ/ህይወት':'ገብረ ህይወት','ገ/ልዑል':'ገብረ ልዑል','ገ/መስቀል':'ገብረ መስቀል','ገ/መድህን':'ገብረ መድህን','ገ/መድኅን':'ገብረ መድኅን','ገ/ሚካኤል':'ገብረ ሚካኤል','ገ/ማሪም':'ገብረ ማሪም','ገ/ማሪያም':'ገብረ ማሪያም','ገ/ማርያም':'ገብረ ማርያም','ገ/ማካኤል':'ገብረ ማካኤል','ገ/ሥላሴ':'ገብረ ሥላሴ','ገ/ስላሴ':'ገብረ ስላሴ','ገ/ተንሳይ':'ገብረ ተንሳይ','ገ/ትንሳይ':'ገብረ ትንሳይ','ገ/አምላክ':'ገብረ አምላክ','ገ/አብ':'ገብረ አብ','ገ/እግዚአብሔር':'ገብረ እግዚአብሔር','ገ/ኪዳን':'ገብረ ኪዳን','ገ/ክርስቶስ':'ገብረ ክርስቶስ','ገ/ክርስቶስና':'ገብረ ክርስቶስና','ገ/ዮሐንስ':'ገብረ ዮሐንስ','ገ/ጊዮርጊስ':'ገብረ ጊዮርጊስ','ገ/ፃዲቅ':'ገብረ ፃዲቅ','ጉ/መ/ቤት':'ጉዳይ መ/ቤት','ጉ/ሚ':'ጉዳይ ሚኒስትር','ጉ/ሚ/ር':'ጉዳይ ሚኒስትር','ጉ/ም/ቤት':'ጉዳይ ምክር ቤት','ጋዜጣና/መጽ':'ጋዜጣና መጽሔት','ግርማ/ልምጭ':'ኃይለ ልምጭ','ግንባር/ኦነግ':'ግንባር /ኦሮሞ ነፃነት ግንባር','ጠ/ሚ':'ጠቅላይ ሚኒስትር','ጠ/ሚ/መለስ':'ጠቅላይ ሚኒስትር መለስ','ጠ/ሚ/መሐመድ':'ጠቅላይ ሚኒስትር መሐመድ','ጠ/ሚ/ሩ':'ጠቅላይ ሚኒስትሩ','ጠ/ሚ/ሩም':'ጠቅላይ ሚኒስትሩም','ጠ/ሚ/ሩስ':'ጠቅላይ ሚኒስትሩስ','ጠ/ሚ/ሩን':'ጠቅላይ ሚኒስትሩን','ጠ/ሚ/ሩንና':'ጠቅላይ ሚኒስትሩንና','ጠ/ሚ/ር':'ጠቅላይ ሚኒስቴር','ጠ/ሚ/ርመለስ':'ጠቅላይ ሚኒስቴር መለስ','ጠ/ሚ/ርነት':'ጠቅላይ ሚኒስትርነት','ጠ/ሚ/ርና':'ጠቅላይ ሚኒስትርና','ጠ/ሚሩ':'ጠቅላይ ሚኒስትሩ','ጠ/ሚሩን':'ጠቅላይ ሚኒስትሩን','ጠ/ሚር':'ጠቅላይ ሚኒስቴር','ጠ/ሚኒስትሩ':'ጠቅላይ ሚኒስትሩ','ጠ/ሚኒስትሩአያይዘውም':'ጠቅላይ ሚኒስትሩ አያይዘውም','ጠ/ሚኒስትር':'ጠቅላይ ሚኒስትር','ጠ/ሚኒስትሮች':'ጠቅላይ ሚኒስትሮች','ጠ/ሚንስትሩ':'ጠቅላይ ሚንስትሩ','ጠ/ሚንስትር':'ጠቅላይ ሚንስትር','ጠ/ም/ቤቱ':'ጠቅላይ ምክር ቤቱ','ጠ/ም/ቤቱን':'ጠቅላይ ምክር ቤቱን','ጠ/ም/ቤታችን':'ጠቅላይ ምክር ቤቶችን','ጠ/ም/ቤት':'ጠቅላይ ምክር ቤት','ጠ/ም/ቤትም':'ጠቅላይ ምክር ቤትም','ጠ/ቤ/ክህነት':'ጠቅላይ ቤተ ክህነት','ጠ/ቤ/ክህነትን':'ጠቅላይ ቤተ ክህነትን','ጠ/ጉባዔ':'ጠቅላይ ጉባዔ','ጠ/ፍ/ቤት':'ጠቅላይ ፍርድ ቤት','ጠ/ፍርድ':'ጠቅላይ ፍርድ','ጽ/ቤ':'ጽህፈት ቤት','ጽ/ቤቱ':'ጽህፈት ቤቱ','ጽ/ቤቱን':'ጽህፈት ቤቱን','ጽ/ቤት':'ጽህፈት ቤት','ጽ/ቤትመግለጫ':'ጽህፈት ቤት መግለጫ','ጽ/ቤትና':'ጽህፈት ቤትና','ጽ/ቤቶች':'ጽህፈት ቤቶች','ፀጋዬጦቢያጋዜጣና/መጽ':'ጦቢያ ጋዜጣና መጽሔት','ፅ/ቤቱ':'ፅህፈት ቤቱ','ፅ/ቤት':'ፅህፈት ቤት','ፌ/ደ/አ/ወ':'ኃላፊነቱ የተወሰነ የግል ማህበር','ፌ/ዴ/ሪ':'ፌደራላዊ ዴሞክራሲ ሪፐብሊክ','ፌ/ፖ':'ፌደራል ፖሊስ','ፍ/ቤቱ':'ፍርድ ቤቱ','ፍ/ቤቱም':'ፍርድ ቤቱም','ፍ/ቤቱን':'ፍርድ ቤቱን','ፍ/ቤቱንና':'ፍርድ ቤቱንና','ፍ/ቤት':'ፍርድ ቤት','ፍ/ቤትም':'ፍርድ ቤትም','ፍ/ቤትን':'ፍርድ ቤትን','ፍ/ቤቶች':'ፍርድ ቤቶች','ፍ/ብ/ችሎት':'ፍትሐ ብሔር ችሎት','ፓርቲ/መኢሶን':'ፓርቲ/መላ ኢትዮጵያ ሶሻሊስት ንቅናቄ','ፓርቲ/ኢዴፓ':'ፓርቲ/ የኢትዮጵያዊያን ዴሞክራሲያዊ ፓርቲ','ፕ/ር':'ፕሮፌሰር','ፕ/ት':'ፕሬዚዳንት','መ/ቅ':'መጸሐፍ ቅዱስ','መ/ቤት':'መሥሪያ ቤት','ቤ/መ':'ቤተመንግስት','ቤ/ክ':'ቤተክርስትያን','አ/አ':'አዲስ አባባ','ዓ/ም':'ዓመተ ምህረት','ዓ/ዓ':'ዓመተ ዓለም',"ዶ/ር":'ዶክተር','ጄ/ል':'ጄኔራል','ጠ/ሚ':'ጠቅላይ ሚኒስትር','ጠ/ሚ/ቢሮ':'ጠቅላይ ሚኒስትር ቢሮ','ጠ/ሚኒስትር':'ጠቅላይ ሚኒስትር','ጽ/ቤት':'ጽህፈት ቤት','ፍ/ቤት':'ፍርድ ቤት','ፕ/ር':'ፕሮፌሰር','ፕ/ት':'ፕሬዚዳንት','ት/ቤት':'ትምህርት ቤት','ወ/ሮ':'ወይዘሮ','ወ/ሪት':'ወይዘሪት','ኮ/ል':'ኮለኔል','ሠ/ፌዴሬሽን':'ሠራተኛ ፌዴሬሽን'}
    indx =0
    for line in train_normalize:
        m=line
        for i, k in enumerate(amh_abbreviation_word):
            for j in line.split(" "):
                if k == j:
                    m = m.replace(k,amh_abbreviation_word[k])
                    train_normalize[indx]=m
        indx =indx +1
    return train_normalize
df['Text'] = AmharicAbbreviationExpansion.AbbreviationExpansion(df['Text']) # Normalization is done on this part

In [ ]:
df= pd.read_csv('/content/expanded_dataset.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

# frequency based normalization

In [ ]:
import pandas as pd
import unicodedata

def normalized(Text):
    normalized = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    ).lower()
    return normalized

# Assuming you have a CSV file with unknown column names
input_csv_path = '/content/drive/MyDrive/expand.csv'
output_csv_path = 'normalized.csv'

# Load the CSV dataset
#df = pd.read_csv(input_csv_path)
df= pd.read_csv('/content/drive/MyDrive/expand.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

# Print the column names to identify the correct column name
print("Column names:", df.Text)

# Check if the specified column name exists in the DataFrame
if 'Amharic_Text' in df.Text:
    # Apply normalization to the 'Amharic_Text' column
    df['Normalized'] = df['Text'].apply(normalized)

    # Save the normalized dataset to a new CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"Normalized saved to {output_csv_path}")
#else:
   ## print("Column 'Amharic_Text' not found in the DataFrame. Please check the actual column names.")


Column names: 0       ቬሮኒካ መንገድ ምግብ ሲሸታት የሸተታት ትገባና ቓጥ ይቅርታ ሪሃና ነብሰጡ...
1                            ቪቫ ከናንተ በወገን ደም መሸቀጥ ስላል ፈለገ
2       ፖሊስ በጀዋር መሃመድ መኖሪያ ህጋዊ ፈቃድ የሌለው ታጣፊ ታንክ በኪስ መያ...
3       ፖለትካ በሃይማኖት ውሰጥ ማንን እንመን አንደ ክላሽ ጠበጃ መንጫ ትላላቹ ...
4       ፌሚኒዝም ከወንዶች አንፃር መቀንቀን የለበትም የእናቱ ሚስቱ እህቱና ልጁ ...
                              ...                        
2731    ሃይ የማይገናኝ አታውራ ታድያ ሰርቶ እንጅ የተከፈለው ሃይማኖቱን ሽጠው ከ...
2732    ሃዋርያት በሰሯት በዘር በቋንቋ በማትከፋፈል በአንድት ሃይማኖት ኦርቶዶክስ...
2733    ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች አበሻ ሴት...
2734              ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች
2735              ሃሳብ ጉንታ ነፍጠኛ ከጠፋ ሌላው ቢኖር እራሱ እንደሌለ ቁጠረው
Name: Text, Length: 2736, dtype: object


In [ ]:
import pandas as pd
import re

def normalize_amharic_text(text):
    char_mapping = {
        'ሐ': 'ሀ',
        'ሓ': 'ሀ',
        'ሃ': 'ሀ',
        'ኀ': 'ሀ',
        'ኃ': 'ሀ',
        'ኻ': 'ሀ',
        'ዅ': 'ሁ',
        'ሑ': 'ሁ',
        'ኁ': 'ሁ',
        'ሒ': 'ሂ',
        'ኂ': 'ሂ',
        'ሔ': 'ሄ',
        'ኄ': 'ሄ',
        'ሕ': 'ህ',
        'ኅ': 'ህ',
        'ሖ': 'ሆ',
        'ኆ': 'ሆ',
        'ሗ': 'ኋ',
        'ዄ': 'ሄ',
        'ኾ': 'ሆ',
        'ሠ': 'ሰ',
        'ሡ': 'ሱ',
        'ሢ': 'ሲ',
        'ሣ': 'ሳ',
        'ሤ': 'ሴ',
        'ሥ': 'ስ',
        'ሦ': 'ሶ',
        'ሼ': 'ሸ',
        'ቼ': 'ቸ',
        'ዬ': 'የ',
        'ዲ': 'ድ',
        'ጄ': 'ጀ',
        'ጸ': 'ፀ',
        'ጹ': 'ፁ',
        'ጺ': 'ፂ',
        'ጻ': 'ፃ',
        'ጼ': 'ፄ',
        'ጽ': 'ፅ',
        'ጾ': 'ፆ',
        'ዓ': 'አ',
        'ዑ': 'ኡ',
        'ዒ': 'ኢ',
        'ዐ': 'አ',
        'ኣ': 'አ',
        'ዔ': 'ኤ',
        'ዕ': 'እ',
        'ዖ': 'ኦ',
        '(ሉ[ዋአ])': 'ሏ',
        '(ሙ[ዋአ])': 'ሟ',
        '(ቱ[ዋአ])': 'ቷ',
        '(ሩ[ዋአ])': 'ሯ',
        '(ሱ[ዋአ])': 'ሷ',
        '(ሹ[ዋአ])': 'ሿ',
        '(ቁ[ዋአ])': 'ቋ',
        '(ቡ[ዋአ])': 'ቧ',
        '(ቹ[ዋአ])': 'ቿ',
        '(ሁ[ዋአ])': 'ኋ',
        '(ኑ[ዋአ])': 'ኗ',
        '(ኙ[ዋአ])': 'ኟ',
        '(ኩ[ዋአ])': 'ኳ',
        '(ዙ[ዋአ])': 'ዟ',
        '(ጉ[ዋአ])': 'ጓ',
        '(ደ[ዋአ])': 'ዷ',
        '(ጡ[ዋአ])': 'ጧ',
        '(ጩ[ዋአ])': 'ጯ',
        '(ጹ[ዋአ])': 'ጿ',
        '(ፉ[ዋአ])': 'ፏ',
        '[ቊ]': 'ቁ',
        '[ኵ]': 'ኩ'
    }

    for char, replacement in char_mapping.items():
        text = text.replace(char, replacement)

    return text
# Assuming you have a CSV file with a column named 'Amharic_Text'
input_csv_path = '/content/drive/MyDrive/expand.csv'
output_csv_path = 'normalized.csv'

# Load the CSV dataset
df= pd.read_csv('/content/drive/MyDrive/expand.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')
# Apply normalization to the 'Amharic_Text' column
df['Normalized_Amharic_Text'] = df['Text'].apply(normalize_amharic_text)

# Save the normalized dataset to a new CSV file
df.to_csv(output_csv_path, index=False)

print(f"Normalized dataset saved to {output_csv_path}")


Normalized dataset saved to normalized.csv


In [ ]:
df= pd.read_csv('/content/normalized.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

In [ ]:
df

,Text,Label,Normalized_Amharic_Text
0,ቬሮኒካ መንገድ ምግብ ሲሸታት የሸተታት ትገባና ቓጥ ይቅርታ ሪሃና ነብሰጡ...,Non-racist,ቬሮኒካ መንገድ ምግብ ሲሸታት የሸተታት ትገባና ቓጥ ይቅርታ ሪሀና ነብሰጡ...
1,ቪቫ ከናንተ በወገን ደም መሸቀጥ ስላል ፈለገ,Non-racist,ቪቫ ከናንተ በወገን ደም መሸቀጥ ስላል ፈለገ
2,ፖሊስ በጀዋር መሃመድ መኖሪያ ህጋዊ ፈቃድ የሌለው ታጣፊ ታንክ በኪስ መያ...,Non-racist,ፖሊስ በጀዋር መሀመድ መኖሪያ ህጋዊ ፈቃድ የሌለው ታጣፊ ታንክ በኪስ መያ...
3,ፖለትካ በሃይማኖት ውሰጥ ማንን እንመን አንደ ክላሽ ጠበጃ መንጫ ትላላቹ ...,Country,ፖለትካ በሀይማኖት ውሰጥ ማንን እንመን አንደ ክላሽ ጠበጃ መንጫ ትላላቹ ...
4,ፌሚኒዝም ከወንዶች አንፃር መቀንቀን የለበትም የእናቱ ሚስቱ እህቱና ልጁ ...,Non-racist,ፌሚኒዝም ከወንዶች አንፃር መቀንቀን የለበትም የእናቱ ሚስቱ እህቱና ልጁ ...
...,...,...,...
2731,ሃይ የማይገናኝ አታውራ ታድያ ሰርቶ እንጅ የተከፈለው ሃይማኖቱን ሽጠው ከ...,Non-racist,ሀይ የማይገናኝ አታውራ ታድያ ሰርቶ እንጅ የተከፈለው ሀይማኖቱን ሽጠው ከ...
2732,ሃዋርያት በሰሯት በዘር በቋንቋ በማትከፋፈል በአንድት ሃይማኖት ኦርቶዶክስ...,Non-racist,ሀዋርያት በሰሯት በዘር በቋንቋ በማትከፋፈል በአንድት ሀይማኖት ኦርቶዶክስ...
2733,ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች አበሻ ሴት...,Country,ሀብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች አበሻ ሴት...
2734,ሃብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች,Country,ሀብታም ስር የማይጠፉ አበሻ ሴቶችና ቆሻሻ የሚርመሰመሱ ዝንቦች


```
# This is formatted as code
```

##Irrelevant charactor removal

In [ ]:
import re

def clean_text(text):
        # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
# Remove URLs
    text = re.sub(r'http\S+http[s]?://(?:[$-_@.&+]|[!*\(\),]|(?:%))+', '', text)
    text = re.sub(r'www\S+', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text
# Apply the clean_text function to the text column of the DataFrame
df['Normalized_Amharic_Text'] = df['Normalized_Amharic_Text'].apply(clean_text)

# ***Translitration***

In [ ]:
import re

# Remove English characters from the 'amharic_text' column of the DataFrame
df['Normalized_Amharic_Text'] = df['Normalized_Amharic_Text'].apply(lambda x: re.sub('[a-zA-Z]', '', x))

In [ ]:
# Save the preprocessed DataFrame to a CSV file
df.to_csv('R-2700.csv', index=False)

In [ ]:
# Save the preprocessed DataFrame to a CSV file
df.to_csv('R-D.xlsx', index=False)

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
df

,Text,Label
0,ጋላን የማዳከሙ ሁኔታ በከፍተኛ የማንቃት ስራ በአመርቂ ሁኔታ ተሰርቷል እ...,Regional
1,ጋላን የማዳከሙ እና የማጥፋት ሁኔታ ህዝብ ከአዛውነት እስከ ህፃናት ሁሉም...,Regional
2,ጋላን የማዳከሙ ሁኔታ የነበልባል ፋኖዎቻችን የታመቀ የፖለቲካ እና የሴራ ...,Individual
3,አማራ ማስ ማድያ የተመሰረተው በነበረከት ቁመት አትፍረዱበት ነፃ አልወጣም...,Non-Racist
4,እነድህ የሚል መፈክር አላቸው ለህብረተሰብ ለውጥ እንተጋለን የአማራ ብዙሀ...,Non-Racist
...,...,...
1130,ኢትዮዽያዊነት ጎንደር ታየች,Individual
1131,በአለም ከምትታወቅበት እራሷን ከውጭ ወረራ ተከላክላ በቀኝ ግዛት ሳትያዝ ...,Individual
1132,ሀቀኛና ደፋር ጋዜጠኛና ሚድያ ቢኖርማ ኖሮ ህዝበ አዳም ሶሻል ሚድያ ተጥዶ...,Individual
1133,ኢትዮጵያዊያን ሙስሊሞች የምንሆንበት የምንጠናክረበት እድል በአላህ ፍቃድ ...,Individual


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/R-D.csv', encoding='utf-8' , delimiter=',')
df= df.fillna(method='ffill')

In [ ]:
df

,Text,Label
0,ጋላን የማዳከሙ በከፍተኛ የማንቃት በአመርቂ ተሰርቷል እየተሰራም,2.0
1,ጋላን የማዳከሙ የማጥፋት ከአዛውነት ህፃናት እውነታውን አውቆ ተስልፏል,2.0
2,ጋላን የማዳከሙ የነበልባል ፋኖዎቻችን የታመቀ የፖለቲካ የሴራ ትንተና እው...,1.0
3,አማራ ማስ ማድያ የተመሰረተው በነበረከት ቁመት አትፍረዱበት ነፃ አልወጣም...,0.0
4,እነድህ የሚል መፈክር አላቸው ለህብረተሰብ ለውጥ እንተጋለን የአማራ ብዙሀ...,0.0
...,...,...
1130,ኢትዮዽያዊነት ጎንደር ታየች,1.0
1131,በአለም ከምትታወቅበት እራሷን ከውጭ ወረራ ተከላክላ በቀኝ ግዛት ሳትያዝ ...,1.0
1132,ሀቀኛና ደፋር ጋዜጠኛና ሚድያ ቢኖርማ ኖሮ ህዝበ አዳም ሶሻል ሚድያ ተጥዶ...,1.0
1133,ኢትዮጵያዊያን ሙስሊሞች የምንሆንበት የምንጠናክረበት እድል በአላህ ፍቃድ ...,1.0
